In [3]:
import cv2
import numpy as np
import math
import skimage
from PIL import Image

def image_preprocessing(img, enhancements):
    # image preprocessing
    gaussian = cv2.GaussianBlur(img, (3,3), cv2.BORDER_DEFAULT)
    gray = cv2.cvtColor(gaussian, cv2.COLOR_BGR2GRAY)
    avg = cv2.blur(gray, (3,3))

    # enhancements: 1 - only contrast enhancement, 2 - only edge enhancement, 3 - both constrast and edge enhancement
    if enhancements == 1:
        # contrast enhancement
        sharp = cv2.equalizeHist(avg)
    elif enhancements == 2:
        # edge enhancement
        gauss = cv2.GaussianBlur(avg, (7,7), 0)
        sharp = cv2.addWeighted(avg, 2, gauss, -1, 0)
    else:
        # contrast enhancement
        he = cv2.equalizeHist(avg)
        
        # edge enhancement
        gauss = cv2.GaussianBlur(he, (7,7), 0)
        sharp = cv2.addWeighted(he, 2, gauss, -1, 0)

    # otsu binarisation
    otsu_threshold, image_result = cv2.threshold(sharp, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    return image_result

def edge_detection(img):
    # edge detection (i)
    edges = cv2.Canny(image=img, threshold1=100, threshold2=200)

    # closing to join gaps between edges (ii)
    kernel = np.ones((5, 5), np.uint8)
    edges = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)
    
    return edges

def contour_detection(img, edges):
    # contour detection
    contours, hierarchy = cv2.findContours(edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    c = max(contours, key = cv2.contourArea)
    approx = cv2.approxPolyDP(c, 0.01*cv2.arcLength(c, True), True)
    
    # Case 1: Rectangular Object
    if len(approx) == 4:
        # draw contour of object
        img = cv2.drawContours(img, [c], -1, (0,255,0), 2)
        
        # circle corners
        for i in range(len(approx)):
            img = cv2.circle(img, approx[i][0], 5, (0,0,255), 3)

        return approx.reshape(-1, 2)
    # Case 2: Irregularly Shaped Object
    else:
        # draw contour of object
        (x,y,w,h) = cv2.boundingRect(c)
        cv2.rectangle(img, (x,y), (x+w, y+h), (0, 255, 0), 2)

        # circle corners
        coords = [
            [x, y],
            [x, y+h],
            [x+w, y],
            [x+w, y+h]
        ]

        for i in range(len(coords)):
            img = cv2.circle(img, coords[i], 5, (0,0,255), 3)
            
        return np.array(coords)
    
def floor_detection(img, coords):
    # get coordinates of pixels that belong to the detected object
    x_coords = [item[0] for item in coords]
    y_coords = [item[1] for item in coords]
    
    # get pixels that are do not belongs to the detected object
    bg = []

    for i in range(img.shape[1]):
        for j in range(img.shape[0]):
            if i in range(min(x_coords), max(x_coords) + 1) and j in range(min(y_coords), max(y_coords) + 1):
                continue
            else:
                bg.append(img[j][i])
                
    # get the number of pixels for each pixel value to determine colour with biggest area in the image, which is assumed as
    # the floor
    unique, counts = np.unique(bg, axis=0, return_counts=True)
            
    # construct a mask for the floor
    upper = unique[np.argmax(counts)] + 10
    lower = unique[np.argmax(counts)] - 10

    mask = cv2.inRange(img, lower, upper)

    # find contours of the floor
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # draw bounding box for the contours of the floor, to be assumed as the floor area
    height, width, _ = img.shape
    min_x, min_y = width, height
    max_x = max_y = 0

    for contour in contours:
        (x,y,w,h) = cv2.boundingRect(contour)
        min_x, max_x = min(x, min_x), max(x+w, max_x)
        min_y, max_y = min(y, min_y), max(y+h, max_y)
    
    return (min_x, min_y, max_x, max_y)

def image_view(floor_coords, img):
    min_x, min_y, max_x, max_y = floor_coords
    height, width, _ = img.shape
    
    bg_w = max_x - min_x
    bg_h = max_y - min_y

    area = bg_w * bg_h
    img_area = height * width

    # img_view: 1 - top down, 2 - side view
    # if detected floor covers more than 90% of the image area, then considered as top down view image
    if (area / img_area) * 100 > 90:
        return 1
    # otherwise, it is considered as side view image
    else:
        return 2
    
# rearrange the order of the coordinates to [top left, top right, bottom right, bottom left]
def order_points(pts):
    rect = np.zeros((4, 2), dtype = "float32")
    
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    
    return rect

# top_axis: 1 - top, 2 - bottom; side_axis: 1 - left, 2 - right
def output_coords(in_coords, top_angle, top_axis, side_angle, side_axis):
    top_left = in_coords[0]
    top_right = in_coords[1]
    bottom_right = in_coords[2]
    bottom_left = in_coords[3]

    # rotation of the image with respect to the specified axis / side, rotates the horizontal view
    if side_angle != 0:
        if side_axis == 1:
            new_top_right = [[], []]
            new_bottom_right = [[], []]

            new_top_right[0] = top_left[0] + ((top_right[0] - top_left[0]) * math.cos(side_angle * math.pi / 180)) + ((top_right[1] - top_left[1]) * math.sin(side_angle * math.pi / 180))
            new_top_right[1] = top_left[1] - ((top_right[0] - top_left[0]) * math.sin(side_angle * math.pi / 180)) + ((top_right[1] - top_left[1]) * math.cos(side_angle * math.pi / 180))
            new_bottom_right[0] = bottom_left[0] + ((bottom_right[0] - bottom_left[0]) * math.cos(side_angle * math.pi / 180)) + ((bottom_right[1] - bottom_left[1]) * math.sin(side_angle * math.pi / 180))
            new_bottom_right[1] = bottom_left[1] - ((bottom_right[0] - bottom_left[0]) * math.sin(side_angle * math.pi / 180)) + ((bottom_right[1] - bottom_left[1]) * math.cos(side_angle * math.pi / 180))

            top_right = new_top_right
            bottom_right = new_bottom_right
        else:
            new_top_left = [[], []]
            new_bottom_left = [[], []]
            
            new_top_left[0] = top_right[0] + ((top_left[0] - top_right[0]) * math.cos(side_angle * math.pi / 180)) + ((top_left[1] - top_right[1]) * math.sin(side_angle * math.pi / 180))
            new_top_left[1] = top_right[1] - ((top_left[0] - top_right[0]) * math.sin(side_angle * math.pi / 180)) + ((top_left[1] - top_right[1]) * math.cos(side_angle * math.pi / 180))
            new_bottom_left[0] = bottom_right[0] + ((bottom_left[0] - bottom_right[0]) * math.cos(side_angle * math.pi / 180)) + ((bottom_left[1] - bottom_right[1]) * math.sin(side_angle * math.pi / 180))
            new_bottom_left[1] = bottom_right[1] - ((bottom_left[0] - bottom_right[0]) * math.sin(side_angle * math.pi / 180)) + ((bottom_left[1] - bottom_right[1]) * math.cos(side_angle * math.pi / 180))
            
            top_left = new_top_left
            bottom_left = new_bottom_left
    
    # rotation of the image with respect to the specified axis / side, rotates the vertical view
    if top_angle != 0:
        if top_axis == 1:
            new_bottom_left = [[], []]
            new_bottom_right = [[], []]

            new_bottom_left[0] = top_left[0] + ((bottom_left[0] - top_left[0]) * math.cos(top_angle * math.pi / 180)) + ((bottom_left[1] - top_left[1]) * math.sin(top_angle * math.pi / 180))
            new_bottom_left[1] = top_left[1] - ((bottom_left[0] - top_left[0]) * math.sin(top_angle * math.pi / 180)) + ((bottom_left[1] - top_left[1]) * math.cos(top_angle * math.pi / 180))
            new_bottom_right[0] = top_right[0] + ((bottom_right[0] - top_right[0]) * math.cos(top_angle * math.pi / 180)) + ((bottom_right[1] - top_right[1]) * math.sin(top_angle * math.pi / 180))
            new_bottom_right[1] = top_right[1] - ((bottom_right[0] - top_right[0]) * math.sin(top_angle * math.pi / 180)) + ((bottom_right[1] - top_right[1]) * math.cos(top_angle * math.pi / 180))

            bottom_left = new_bottom_left
            bottom_right = new_bottom_right
        else:
            new_top_left = [[], []]
            new_top_right = [[], []]
            
            new_top_left[0] = bottom_left[0] + ((top_left[0] - bottom_left[0]) * math.cos(top_angle * math.pi / 180)) + ((top_left[1] - bottom_left[1]) * math.sin(top_angle * math.pi / 180))
            new_top_left[1] = bottom_left[1] - ((top_left[0] - bottom_left[0]) * math.sin(top_angle * math.pi / 180)) + ((top_left[1] - bottom_left[1]) * math.cos(top_angle * math.pi / 180))
            new_top_right[0] = bottom_right[0] + ((top_right[0] - bottom_right[0]) * math.cos(top_angle * math.pi / 180)) + ((top_right[1] - bottom_right[1]) * math.sin(top_angle * math.pi / 180))
            new_top_right[1] = bottom_right[1] - ((top_right[0] - bottom_right[0]) * math.sin(top_angle * math.pi / 180)) + ((top_right[1] - bottom_right[1]) * math.cos(top_angle * math.pi / 180))

            top_left = new_top_left
            top_right = new_top_right

    return [[top_left, top_right, bottom_right, bottom_left]]

# get coordinates to unwarp image into centered view
def dest_coords(shape):
    dst = np.zeros((4, 2), dtype = "float32")
    
    dst[0] = [0, 0]
    dst[1] = [shape[1], 0]
    dst[2] = [shape[1], shape[0]]
    dst[3] = [0, shape[0]]
    
    return dst

# get perspective transformed image according to defined angles
# int_flag: 0 - nearest, 1 - linear, 2 - cubic
def perspectiveTransform(img, in_coords, out_coords, int_flag):
    query_pts = np.float32(in_coords)
    train_pts = np.float32(out_coords)

    matrix = cv2.getPerspectiveTransform(query_pts, train_pts)
    
    x_coords = [item[0] for item in out_coords]
    y_coords = [item[1] for item in out_coords]
    
    output_size = (math.ceil(max(img.shape[1], max(x_coords))), math.ceil(max(img.shape[0], max(y_coords))))

    dst = cv2.warpPerspective(img, matrix, output_size, flags=int_flag)
    
    return dst

# function for PIL to find transformation coefficients
def find_coeffs(pa, pb):
    matrix = []
    for p1, p2 in zip(pa, pb):
        matrix.append([p1[0], p1[1], 1, 0, 0, 0, -p2[0]*p1[0], -p2[0]*p1[1]])
        matrix.append([0, 0, 0, p1[0], p1[1], 1, -p2[1]*p1[0], -p2[1]*p1[1]])

    A = np.matrix(matrix, dtype=np.float32)
    B = np.array(pb).reshape(8)

    res = np.dot(np.linalg.inv(A.T * A) * A.T, B)
    return np.array(res).reshape(8)

In [37]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
from tkinter import ttk

coords = []
count = 0
img_pts = 0
reselect = False

def get_img():
    coords = []
    count = 0
    reselect = False
    
    path = filedialog.askopenfilename(title='Select Image')
    
    return path

def save_img():
    path = filedialog.asksaveasfile(initialfile='transformed_img', defaultextension=".png", filetypes=[("All Files","*.*"),("Image Files","*.png")])
    
    return path.name

def select_new_pts(event, x, y, flags, param):
    global coords, count
    if event == cv2.EVENT_LBUTTONDOWN:
        coords.append((x, y))
        cv2.circle(img_pts, (x, y), 5, (0,255,255), 3)
        count += 1

def redefine_points(path, detImageLabel):
    tk.messagebox.showinfo("Redefine Points", "Left click to select a point, window will close automatically upon selection of 4 points. \
    Please specify your points in clockwise order, starting from the top left point, to ensure a correct output is obtained. \
    To stop selecting points, press on 'esc' and none of the points selected will be used. ")

    global img_pts, coords, count, reselect
    
    coords = []
    count = 0
    img_pts = cv2.imread(path)

    cv2.namedWindow("Select Points")

    cv2.setMouseCallback("Select Points", select_new_pts)

    while True:
        cv2.imshow("Select Points", img_pts)
        if cv2.waitKey(10) == 27 or count == 4:
            reselect = True
            break

    cv2.destroyAllWindows()
    
    if len(coords) == 4:
        image = cv2.polylines(img_pts, np.int32([coords]), True, (255, 0, 0), 2)
        
        for i in range(len(coords)):
            cv2.circle(img_pts, (int(coords[i][0]), int(coords[i][1])), 5, (0,0,255), 3)
        
        imgDet = img_pts.copy()
        imgDet = cv2.cvtColor(imgDet, cv2.COLOR_BGR2RGB)
        imgDet = Image.fromarray(np.uint8(imgDet))
        
        if (img_pts.shape[1] > img_pts.shape[0]):
            resizedDet = imgDet.resize((383, 262))
        else:
            resizedDet = imgDet.resize((267, 350))

        tkimageDet = ImageTk.PhotoImage(resizedDet)
        
        detImageLabel.configure(image=tkimageDet)
        detImageLabel.image = tkimageDet
        
        reselect = True

def enable_interpolation(interpolation_btns):
    for btn in interpolation_btns:
        btn.configure(state = 'enabled')

def disable_interpolation(interpolation_btns):
    for btn in interpolation_btns:
        btn.configure(state = 'disabled')

def clear_window():
    for widget in window.winfo_children():
        widget.destroy()

def display_main():
    clear_window()
    window.update_idletasks()
    
    title = tk.Label(window, text = "Welcome to Perspective Transformation System!", fg = "black", bg = "white", 
                     font = ("Arial", 20))
    title.place(x = 155, y = 150)

    file = tk.Button(window, text = "Choose Image", command = lambda: display_image(get_img()), height = 2, width = 15)
    file.place(x = 250, y = 300)

    close = tk.Button(window, text = "Exit", command = window.destroy, height = 2, width = 15)
    close.place(x = 500, y = 300)
    
# trans_type: 1 - straighten, 2 - transform
def display_transformed(path, processing, library, interpolation, degree_vertical, position_vertical, degree_horizontal, position_horizontal, obj_coords, trans_type):
    clear_window()
    
    imgCV = cv2.imread(path)
    
    if reselect == True:
        in_coords = np.array(coords).reshape(-1, 2)
    else:
        in_coords = order_points(obj_coords)
    
    if trans_type == 1:
        out_coords = dest_coords(imgCV.shape)
    else:
        if position_vertical == "Top":
            top_ax = 1
        else:
            top_ax = 2
            
        if position_horizontal == "Left":
            side_ax = 1
        else:
            side_ax = 2
        
        out_coords = np.array(output_coords(in_coords, degree_vertical, top_ax, degree_horizontal, side_ax)).reshape(-1, 2)

    if library == "OpenCV":
        if interpolation == "Nearest Neighbor":
            int_flag = 0
        elif interpolation == "Bilinear":
            int_flag = 1
        elif interpolation == "Bicubic":
            int_flag = 2
        elif interpolation == "Pixel Area":
            int_flag = 3
        else:
            int_flag = 4

        transformed_image = perspectiveTransform(imgCV, in_coords, out_coords, int_flag)
    elif library == "Pillow":
        imgCV_copy = imgCV.copy()
        imgCV_copy = cv2.cvtColor(imgCV_copy, cv2.COLOR_BGR2RGB)
        imgCV_copy = Image.fromarray(np.uint8(imgCV_copy))

        width, height = imgCV_copy.size
        x_coords = [item[0] for item in out_coords]
        y_coords = [item[1] for item in out_coords]
        
        coeffs = find_coeffs(np.array(in_coords), np.array(out_coords).reshape(-1, 2))
        output_size = (math.ceil(max(imgCV.shape[1], max(x_coords))), math.ceil(max(imgCV.shape[0], max(y_coords))))
        transformed_image = imgCV_copy.transform(output_size, Image.PERSPECTIVE, coeffs)
    else:
        x_coords = [item[0] for item in out_coords]
        y_coords = [item[1] for item in out_coords]

        tform3 = skimage.transform.ProjectiveTransform()
        tform3.estimate(np.array(in_coords), np.array(out_coords).reshape(-1, 2))
        output_size = (math.ceil(max(imgCV.shape[1], max(x_coords))), math.ceil(max(imgCV.shape[0], max(y_coords))))
        transformed_image = skimage.transform.warp(imgCV, tform3, output_shape=(output_size[1], output_size[0]))
    
    oriLabelDT = tk.Label(window, text = "Original Image", font = ("Arial", 15), bg="white", fg="black")
    oriLabelDT.place(x = 170, y = 20)
    
    transLabelDT = tk.Label(window, text = "Transformed Image", font = ("Arial", 15), bg="white", fg="black")
    transLabelDT.place(x = 580, y = 20)
    
    imgTransDT = transformed_image.copy()
    
    if library == "OpenCV" or library == "Scikit":
        if library == "Scikit":
            imgTransDT = skimage.img_as_ubyte(imgTransDT)
            
        imgTransDT = cv2.cvtColor(imgTransDT, cv2.COLOR_BGR2RGB)
        imgTransDT = Image.fromarray(np.uint8(imgTransDT))
    
    imgDT = Image.open(path)
    photoImgDT = ImageTk.PhotoImage(imgDT)

    if (photoImgDT.width() > photoImgDT.height()):
        resizedDT = imgDT.resize((383, 262))
        resizedTransDT = imgTransDT.resize((383, 262))
        xpos = 45
        ypos = 100
    else:
        resizedDT = imgDT.resize((267, 350))
        resizedTransDT = imgTransDT.resize((267, 350))
        xpos = 110
        ypos = 50

    tkimageOriDT = ImageTk.PhotoImage(resizedDT)
    oriImgLabelDT = tk.Label(window, image = tkimageOriDT)
    oriImgLabelDT.image = tkimageOriDT
    oriImgLabelDT.place(x = xpos, y = ypos)
    
    tkimageTransDT = ImageTk.PhotoImage(resizedTransDT)
    transLabelDT = tk.Label(window, image = tkimageTransDT)
    transLabelDT.image = tkimageTransDT
    transLabelDT.place(x = xpos + 430, y = ypos)

    window.update_idletasks()
    saveDT = tk.Button(window, text = "Save Output Image", command = lambda: cv2.imwrite(save_img(), transformed_image), height = 2, width = 15)
    saveDT.place(x = 180, y = 455)
    
    retDT = tk.Button(window, text = "Return", command = lambda: display_detection(path, processing, library, interpolation, degree_vertical, position_vertical, degree_horizontal, position_horizontal, trans_type), height = 2, width = 15)
    retDT.place(x = 325, y = 455)
    
    homeDT = tk.Button(window, text = "Home", command = display_main, height = 2, width = 15)
    homeDT.place(x = 470, y = 455)

    closeDT = tk.Button(window, text = "Exit", command = window.destroy, height = 2, width = 15)
    closeDT.place(x = 615, y = 455)

def display_detection(path, processing, library, interpolation, degree_vertical, position_vertical, degree_horizontal, position_horizontal, trans_type):
    clear_window()
    
    global coords, reselect
    coords = []
    reselect = False
    
    imgCV = cv2.imread(path)
    
    if processing == "Contrast":
        preprocessed_img = image_preprocessing(imgCV, 1)
    elif processing == "Edge":
        preprocessed_img = image_preprocessing(imgCV, 2)
    else:
        preprocessed_img = image_preprocessing(imgCV, 3)
        
    edges = edge_detection(preprocessed_img)
    
    obj_coords = contour_detection(imgCV, edges)
    
    floor_coords = floor_detection(imgCV, obj_coords)
    im_view = image_view(floor_coords, imgCV)
    
    if im_view == 1:
        im_view_text = "Image View: Top Down"
    else:
        im_view_text = "Image View: Side View"
    
    oriLabel = tk.Label(window, text = "Original Image", font = ("Arial", 15), bg="white", fg="black")
    oriLabel.place(x = 170, y = 20)
    
    detLabel = tk.Label(window, text = "Detected Object", font = ("Arial", 15), bg="white", fg="black")
    detLabel.place(x = 600, y = 20)
    
    viewLabel = tk.Label(window, text = im_view_text, font = ("Arial", 12), bg="white", fg="black")
    viewLabel.place(x = 150, y = 410)
    
    imgDet = imgCV.copy()
    imgDet = cv2.cvtColor(imgDet, cv2.COLOR_BGR2RGB)
    imgDet = Image.fromarray(np.uint8(imgDet))
    
    img = Image.open(path)
    photoImg = ImageTk.PhotoImage(img)

    if (photoImg.width() > photoImg.height()):
        resizedOri = img.resize((383, 262))
        resizedDet = imgDet.resize((383, 262))
        xpos = 45
        ypos = 100
    else:
        resizedOri = img.resize((267, 350))
        resizedDet = imgDet.resize((267, 350))
        xpos = 110
        ypos = 50

    tkimageOri = ImageTk.PhotoImage(resizedOri)
    oriImgLabel = tk.Label(window, image = tkimageOri)
    oriImgLabel.image = tkimageOri
    oriImgLabel.place(x = xpos, y = ypos)
    
    tkimageDet = ImageTk.PhotoImage(resizedDet)
    detImgLabel = tk.Label(window, image = tkimageDet)
    detImgLabel.image = tkimageDet
    detImgLabel.place(x = xpos + 430, y = ypos)
    
    proceed = tk.Button(window, text = "Proceed", command = lambda: display_transformed(path, processing, library, interpolation, degree_vertical, position_vertical, degree_horizontal, position_horizontal, obj_coords, trans_type), height = 2, width = 15)
    proceed.place(x = 110, y = 455)
    
    ret = tk.Button(window, text = "Return", command = lambda: display_image(path), height = 2, width = 15)
    ret.place(x = 260, y = 455)
    
    redef = ret = tk.Button(window, text = "Redefine Points", command = lambda: redefine_points(path, detImgLabel), height = 2, width = 15)
    redef.place(x = 410, y = 455)
    
    home = tk.Button(window, text = "Home", command = display_main, height = 2, width = 15)
    home.place(x = 555, y = 455)

    close = tk.Button(window, text = "Exit", command = window.destroy, height = 2, width = 15)
    close.place(x = 700, y = 455)
    
def display_image(path):
    clear_window()
    
    if path == '':
        empty = tk.Label(window, text = "No Image Chosen", fg = "black", bg = "white", font = ("Arial", 15))
        empty.place(x = 215, y = 200)
    else:
        img = Image.open(path)
        photoImg = ImageTk.PhotoImage(img)

        if (photoImg.width() > photoImg.height()):
            resized = img.resize((510, 349))
            xpos = 50
        else:
            resized = img.resize((267, 350))
            xpos = 160

        tkimage = ImageTk.PhotoImage(resized)
        imgLabel = tk.Label(window, image = tkimage)
        imgLabel.image = tkimage
        imgLabel.place(x = xpos, y = 50)
    
    file = tk.Button(window, text = "Choose Image", command = lambda: display_image(get_img()), height = 2, width = 15)
    file.place(x = 100, y = 425)
    
    ret = tk.Button(window, text = "Return", command = display_main, height = 2, width = 15)
    ret.place(x = 245, y = 425)

    close = tk.Button(window, text = "Exit", command = window.destroy, height = 2, width = 15)
    close.place(x = 390, y = 425)
    
    # Separator object
    separator = ttk.Separator(window, orient='vertical')
    separator.place(x=590, y=0, width=0.2, relheight=1)
    
    canvasFrame = tk.Frame(window, bg="white")
    canvasFrame.place(x=591, y=0)

    canvas = tk.Canvas(canvasFrame, bg = "white", width=288, height=520)
    canvas.grid(row = 0, column = 0, sticky = "news")

    scrollbar = tk.Scrollbar(canvasFrame, orient = "vertical", command = canvas.yview)
    scrollbar.grid(row = 0, column = 1, sticky = "ns")
    canvas.configure(yscrollcommand=scrollbar.set)

    ctrlFrame = tk.Frame(canvas, bg = "white")
    canvas.create_window((0, 0), window = ctrlFrame, anchor = "nw")
    
    ctrlLabel = tk.Label(ctrlFrame, text = "Controls", fg = "black", bg = "white", font = ("Arial", 13))
    ctrlLabel.grid(row=0, column=0, pady=10)
    
    #================================= Image Preprocessing =================================
    image_processing_frame = tk.Frame(ctrlFrame, highlightthickness=1, highlightbackground="#989898")
    image_processing_label = tk.Label(image_processing_frame, text="Image Preprocessing")
    image_processing_label.pack(pady=5)

    edge_button = ttk.Radiobutton(image_processing_frame, text="Edge", variable=processing_var, value="Edge")
    contrast_button = ttk.Radiobutton(image_processing_frame, text="Contrast", variable=processing_var, value="Contrast")
    edge_contrast_button = ttk.Radiobutton(image_processing_frame, text="Edge+Contrast", variable=processing_var, value="Edge+Contrast")
    edge_button.pack(side="left", padx=5)
    contrast_button.pack(side="left", padx=5)
    edge_contrast_button.pack(side="left", padx=5)
    image_processing_frame.grid(row=1, column=0, padx=18, pady=(0,10))
    
    #================================= Library =================================
    library_frame = tk.Frame(ctrlFrame, width=250, height=60, highlightthickness=1, highlightbackground="#989898")
    library_label = tk.Label(library_frame, text="Library")
    library_label.pack(pady=5)

    opencv_button = ttk.Radiobutton(library_frame, text="OpenCV", variable=library_var, value="OpenCV")
    scikit_button = ttk.Radiobutton(library_frame, text="Scikit", variable=library_var, value="Scikit")
    pillow_button = ttk.Radiobutton(library_frame, text="Pillow", variable=library_var, value="Pillow")
    opencv_button.pack(side="left", padx=(25, 5))
    scikit_button.pack(side="left", padx=5)
    pillow_button.pack(side="left", padx=5)
    library_frame.grid(row=2, column=0, padx=18, pady=10)
    library_frame.pack_propagate(0)
    
    #================================= Interpolation =================================
    # Frame for Interpolation
    interpolation_frame = ttk.Frame(ctrlFrame, width=250, height=100, borderwidth=2, relief="groove")
    interpolation_label = tk.Label(interpolation_frame, text="Interpolation")
    interpolation_label.grid(row=0, column=0, columnspan=2, padx=90)

    nearest_neighbor_button = ttk.Radiobutton(interpolation_frame, text="Nearest Neighbor", variable=interpolation_var, value="Nearest Neighbor")
    bilinear_button = ttk.Radiobutton(interpolation_frame, text="Bilinear", variable=interpolation_var, value="Bilinear")
    bicubic_button = ttk.Radiobutton(interpolation_frame, text="Bicubic", variable=interpolation_var, value="Bicubic")
    pxarea_button = ttk.Radiobutton(interpolation_frame, text="Pixel Area", variable=interpolation_var, value="Pixel Area")
    lanczos_button = ttk.Radiobutton(interpolation_frame, text="Lanczos", variable=interpolation_var, value="Lanczos")
    nearest_neighbor_button.grid(row=1, column=0, sticky='w', padx=(5, 0))
    bilinear_button.grid(row=2, column=0, sticky='w', padx=(5, 0))
    bicubic_button.grid(row=3, column=0, sticky='w', padx=(5, 0))
    pxarea_button.grid(row=1, column=1, sticky='w')
    lanczos_button.grid(row=2, column=1, sticky='w')
    interpolation_frame.grid(row=3, column=0, padx=18, pady=10)
    interpolation_frame.grid_propagate(0)
    
    opencv_button.configure(command=lambda: enable_interpolation([nearest_neighbor_button, bilinear_button, bicubic_button, pxarea_button, lanczos_button]))
    scikit_button.configure(command=lambda: disable_interpolation([nearest_neighbor_button, bilinear_button, bicubic_button, pxarea_button, lanczos_button]))
    pillow_button.configure(command=lambda: disable_interpolation([nearest_neighbor_button, bilinear_button, bicubic_button, pxarea_button, lanczos_button]))
    
    #================================= Vertical Controls (L) =================================
    vertical_group_frame = tk.Frame(ctrlFrame, width=250, height=185, highlightthickness=1, highlightbackground="#989898")
    vertical_group_label = tk.Label(vertical_group_frame, text="Vertical Rotation", font=("Helvetica", 10))
    vertical_group_label.pack(pady=(10, 10))
    degree_label_vertical = tk.Label(vertical_group_frame, text="Vertical Degree:")
    degree_scale_vertical = tk.Scale(vertical_group_frame, variable=degree_var_vertical, from_=-90, to=90, resolution=1, orient="horizontal")
    degree_textbox_vertical = tk.Entry(vertical_group_frame, textvariable=degree_var_vertical)

    # Radio buttons for Vertical Position
    position_frame_vertical = ttk.Frame(vertical_group_frame)
    position_label_vertical = tk.Label(position_frame_vertical, text="Axis:")
    top_button_vertical = ttk.Radiobutton(position_frame_vertical, text="Top", variable=position_var_vertical, value="Top")
    bottom_button_vertical = ttk.Radiobutton(position_frame_vertical, text="Bottom", variable=position_var_vertical, value="Bottom")

    # Pack the widgets for vertical degree and position controls
    vertical_group_label.pack()
    degree_label_vertical.pack()
    degree_scale_vertical.pack()
    degree_textbox_vertical.pack()
    position_label_vertical.pack()
    top_button_vertical.pack(side="left")
    bottom_button_vertical.pack(side="left")
    position_frame_vertical.pack()
    vertical_group_frame.grid(row=4, column=0, padx=18, pady=10)
    vertical_group_frame.pack_propagate(0)

    #================================= Horizontal Controls (R) =================================
    horizontal_group_frame = tk.Frame(ctrlFrame, width=250, height=185, highlightthickness=1, highlightbackground="#989898")
    horizontal_group_label = tk.Label(horizontal_group_frame, text="Horizontal Rotation", font=("Helvetica", 10))
    horizontal_group_label.pack(pady=(10, 10))
    degree_label_horizontal = tk.Label(horizontal_group_frame, text="Horizontal Degree:")
    degree_scale_horizontal = tk.Scale(horizontal_group_frame, variable=degree_var_horizontal, from_=-90, to=90, resolution=1, orient="horizontal")
    degree_textbox_horizontal = tk.Entry(horizontal_group_frame, textvariable=degree_var_horizontal)

    # Radio buttons for Horizontal Position
    position_frame_horizontal = ttk.Frame(horizontal_group_frame)
    position_label_horizontal = tk.Label(position_frame_horizontal, text="Axis:")
    left_button_horizontal = ttk.Radiobutton(position_frame_horizontal, text="Left", variable=position_var_horizontal, value="Left")
    right_button_horizontal = ttk.Radiobutton(position_frame_horizontal, text="Right", variable=position_var_horizontal, value="Right")

    # Pack the widgets for horizontal degree and position controls
    horizontal_group_label.pack()
    degree_label_horizontal.pack()
    degree_scale_horizontal.pack()
    degree_textbox_horizontal.pack()
    position_label_horizontal.pack()
    left_button_horizontal.pack(side="left")
    right_button_horizontal.pack(side="left")
    position_frame_horizontal.pack(pady=5)
    horizontal_group_frame.grid(row=5, column=0, padx=18, pady=10)
    horizontal_group_frame.pack_propagate(0)
    
    #================================= Transform Button =================================
    transform_button_frame = tk.Frame(ctrlFrame, bg="white")
    straight_im_buttom = tk.Button(transform_button_frame, text="Straighten Image", relief=tk.RAISED)
    transform_button = tk.Button(transform_button_frame, text="Transform Image", relief=tk.RAISED)
    straight_im_buttom.pack(side="left", padx=(0, 25))
    transform_button.pack(side="right")
    transform_button_frame.grid(row=6, column=0, pady=(10, 20))
    
    straight_im_buttom.configure(command=lambda: display_detection(path, processing_var.get(), library_var.get(), interpolation_var.get(), degree_var_vertical.get(), position_var_vertical.get(), degree_var_horizontal.get(), position_var_horizontal.get(), 1) if path != '' else tk.messagebox.showerror('Image Error', 'Error: No Image to Transform!'))
    transform_button.configure(command=lambda: display_detection(path, processing_var.get(), library_var.get(), interpolation_var.get(), degree_var_vertical.get(), position_var_vertical.get(), degree_var_horizontal.get(), position_var_horizontal.get(), 2) if path != '' else tk.messagebox.showerror('Image Error', 'Error: No Image to Transform!'))
    
    ctrlFrame.update_idletasks()

    canvas.config(scrollregion = canvas.bbox("all"))
    canvas.bind_all("<MouseWheel>", lambda event: canvas.yview_scroll(int(-1*(event.delta/120)), "units"))

window = tk.Tk()
window.title("Perspective Transformation System")
window.geometry("900x520")
window.configure(bg = "white")

processing_var = tk.StringVar(value="Edge")
interpolation_var = tk.StringVar(value="Nearest Neighbor")
library_var = tk.StringVar(value="OpenCV")
degree_var_vertical = tk.DoubleVar()
position_var_vertical = tk.StringVar(value="Top")
degree_var_horizontal = tk.DoubleVar()
position_var_horizontal = tk.StringVar(value="Left")

display_main()

window.mainloop()